In [5]:
import os
from dotenv import load_dotenv
import pandas as pd
import pickle
import numpy as np
import os
import sys

sys.path.append(os.path.dirname(os.getcwd()))
from groq import Groq
from src.llm_engine.gemini_agent import Agent


In [18]:
load_dotenv()

True

In [6]:
# client = Groq(
#     api_key= os.getenv("GROQ_API_KEY"),
# )

client = Groq(api_key= os.getenv("GROQ_API_KEY"))

In [7]:
df = pd.read_csv(r"data/GridPuzzle_processed.csv")

df.head()

,key,id,question,answer,Mistral-7b,Llama-13b,gemini-pro,gpt-4-turbo,Claude-3,actual_question,Clues,categories,number_of_clues,number_categories,category_names
0,341.json,802,Nicholas and Edith are having a small dinner p...,1984 | Annata Branco | gewurztraminer\n1988 | ...,| 1984 | Annata Branco | gewurztraminer |\n |-...,Let's solve the puzzle step by step.\n \n Step...,"Step-by-step solution:\n 1. From clue 6, we kn...","Step-by-step solution:\n \n 1. From clue 6, we...","Step-by-step solution:\n \n 1. From clue 6, we...",Nicholas and Edith are having a small dinner p...,Clues:\n1. The Ece Suss was bottled sometime a...,"vintages : 1984, 1988, 1992, 1996.\nwines : An...",6,3,vintages wines types
1,341.json,726,The world-famous Punta Santiago Rose Gardening...,first | bon silene | Renee\nsecond | perle d'o...,| first | calocarpa | Hannah\n |---------|----...,Let's break down the clues step by step:\n \n ...,"Step-by-step solution:\n 1. From clue 1, we kn...","Step-by-step solution:\n \n 1. From clue 3, we...","Step-by-step solution:\n \n 1. From clue 1, we...",The world-famous Punta Santiago Rose Gardening...,Clues:\n1. The rose that won fourth place was ...,"placements : first, second, third, fourth.\nro...",5,3,placements roses gardeners
2,341.json,16,A standardized test has recently been develope...,55% | Archimedes | CIT\n62% | Galen | UCLA\n69...,| 55% | Zeno | CIT |\n |------|------|-----|\n...,Let's break down the clues and solve the puzzl...,"Step-by-step solution:\n 1. From clue 2, we kn...","Step-by-step solution:\n \n 1. From clue 4, Pr...","Step-by-step solution:\n \n 1. From clue 4, we...",A standardized test has recently been develope...,Clues:\n1. Zeno finished with a score that was...,"scores : 55%, 62%, 69%, 76%.\na.i. : Archimede...",5,2,scores institutions
3,341.json,698,Gil Glabbers has lived on the coast of Guernse...,1962 | Tracy | Cherbourg\n1969 | Felipe | Bour...,| 1962 | Felipe (Bournemouth) |\n | 1969 | Kar...,Let's break down the clues step by step:\n \n ...,"Step-by-step solution:\n \n 1. From clue 2, we...","Step-by-step solution:\n \n 1. From Clue 2, Tr...","Step-by-step solution:\n \n 1. From clue 2, we...",Gil Glabbers has lived on the coast of Guernse...,Clues:\n1. The message from Plymouth was sent ...,"year sent : 1962, 1969, 1976, 1983.\nwriters :...",5,3,sent writers origins
4,341.json,444,Jamie never misses an opportunity to watch the...,2 | April 22 | Eastbrook\n9 | April 10 | Islet...,| Dates | Locations |\n |---|---|\n | April 10...,Let's break down the clues and solve the puzzl...,"Step-by-step solution:\n \n 1. From clue 1, we...",Step-by-step solution:\n \n Step 1: Analyze th...,"Step-by-step solution:\n \n 1. From clue 3, we...",Jamie never misses an opportunity to watch the...,Clues:\n1. The outing to Eastbrook logged 7 fe...,"shooting stars : 2, 9, 16, 23.\ndates : April ...",4,3,stars dates locations


In [8]:
import re
def process_ls(ls):
    if re.search(r"\d+",ls[0]) is None:
        return ls

    else:
        res = []
        for val in ls:
            val = re.sub(r"[^\d+]", '', val)
            res.append(val)
        return res


In [9]:
import re
def process_question(question):
    lines = question.split("\n")
    categories = {}
    clues = []
    actual_question = ""
    ckpt = -1
    flag = -1
    for index, line in enumerate(lines):


        if ckpt == 1 and re.search(r"^\d+\.", line) is None:
            break

        if " : " in line:
            if flag == -1:
                flag = index
            title, ls = line.split(" : ")[0], line.split(" : ")[1].split(", ")
            ls = process_ls(ls)
            categories[title] = ls
        
        if "Clues" in line:
            ckpt = 1
            continue
        
        if ckpt == 1 and re.search(r"\d+\.", line) is not None:
            clues.append(line)
        


    actual_question += "\n".join(lines[:flag])            

    return actual_question, categories, clues


#ques, categories, clues = process_question(df["question"].iloc[100])


In [6]:
ind = 12
ques, categories, clues = process_question(df["question"].iloc[ind])


In [10]:
system_prompt = f"""

You are an expert AI assistant for analyzing logic grid puzzles. Your task is to systematically translate puzzle clues into logical expressions.
Analyze each clue and categories input to frame a sequence of logical expressions.

Remember to carefully comprehend each clue step by step and translate them to logical expressions
Retun ONLY a thoughtful sequence of logical expressions.

"""


agent = Agent(system_prompt=system_prompt, agent_name="csp_agent")
user_query= f"""{ques}\n\n{clues}\n\n{categories}"""

agent_1_response = agent.perform_action(user_query)


NameError: name 'ques' is not defined

In [8]:
print(agent_1_response)

Let's denote the months as J, F, M, A; probes as D, I, MJ, VM; and destinations as G, Me, P, U.

1. "The Iunsel launched sometime after the probe going to Mercury."
   I > Me

2. "The Disty launched in March."
   D = M

3. "The Moro Jaddo launched 1 month after the mission going to Uranus."
   MJ = U + 1

4. "The Iunsel is going to Pluto."
   I = P

5. "The Iunsel launched 1 month before the mission going to Uranus."
   I = U - 1

Combining 3 and 5, we have:
MJ = U + 1 and I = U - 1, therefore MJ = I + 2

From clue 2, D = M.
From clue 5, I = U - 1. Since months are J, F, M, A, U cannot be J.
From clue 3, MJ = U + 1. Since months are J, F, M, A, U cannot be A.

From clue 1, I > Me.

Let's analyze the possible months for I and U:
If I = J, then U = F. MJ = M. But D = M, so this is not possible.
If I = F, then U = M. MJ = A.
If I = M, then U = A. MJ would be after April, which is not possible.

Therefore, I = F, U = M, MJ = A.
Since D = M, and U = M, this is not possible.

Let's re-examin

In [8]:
print(agent_1_response)

Let's denote the categories as follows:
*   Prices: P = {125, 150, 175, 200, 225}
*   Customers: C = {Erica, Francisco, Heidi, Joey, Zachary}
*   Shapes: S = {beach chair, football, golf ball, sports car, train}
*   Flavors: F = {banana, chocolate, peach, raspberry, strawberry}

Now, let's translate the clues into logical expressions:

1.  Zachary's cake isn't strawberry-flavored.
    *   ¬(Zachary = strawberry)

2.  The $150 cake is either Erica's piece or the raspberry-flavored cake.
    *   150 = Erica ∨ 150 = raspberry

3.  Francisco's piece isn't banana-flavored.
    *   ¬(Francisco = banana)

4.  The chocolate-flavored cake costs 25 dollars less than the cake shaped like a sports car.
    *   chocolate = sports car - 25

5.  Of the peach-flavored cake and the cake shaped like a golf ball, one is Erica's and the other is Joey's.
    *   (peach = Erica ∧ golf ball = Joey) ∨ (peach = Joey ∧ golf ball = Erica)

6.  The $125 piece is strawberry-flavored.
    *   125 = strawberry

7.  

In [9]:
user_query = """You are an expert AI assistant for analyzing logical constraints. Analyze the constraints given below and 
map the categories accordingly.
"""
cat_str = """"""
for cat in categories:

    cat_str += f"""{cat}: {", ".join(categories[cat])}\n"""


user_query += cat_str


user_query += agent_1_response + "\n"

return_seq = df.question.iloc[ind].split("Final Answer:")[1]

user_query += return_seq + "\n"

completion = client.chat.completions.create(
    model="deepseek-r1-distill-llama-70b",
    messages=[{
            "role": "user",
            "content": user_query

        }],
    temperature=0,
    max_completion_tokens=8192,
    top_p=1,
    stream=True,
    stop=None,
)

groq_deepseek_response = ""
for chunk in completion:
    text = chunk.choices[0].delta.content or ""
    groq_deepseek_response += text

print(groq_deepseek_response)

<think>
Okay, let's try to solve this problem step by step. I'm a bit confused, but I'll take it slowly.

First, let's list out all the given information:

Months: January (J), February (F), March (M), April (A)
Probes: Disty (D), Iunsel (I), Moro Jaddo (MJ), Viria Mari (VM)
Destinations: Ganymede (G), Mercury (Me), Pluto (P), Uranus (U)

Clues:
1. I > Me (Iunsel launched after the probe going to Mercury)
2. D = M (Disty launched in March)
3. MJ = U + 1 (Moro Jaddo launched 1 month after the mission to Uranus)
4. I = P (Iunsel is going to Pluto)
5. I = U - 1 (Iunsel launched 1 month before the mission to Uranus)

From clue 4, I = P, so Iunsel's destination is Pluto.

From clue 5, I = U - 1, which means Iunsel launched one month before Uranus mission. So if I is in month X, U is in X+1.

From clue 3, MJ = U + 1, so Moro Jaddo launched one month after Uranus mission. So MJ is in (U +1).

Combining clues 3 and 5: Since I = U -1, then MJ = (U) +1 = (I +1) +1 = I +2. So MJ is two months aft

In [10]:
print(df.answer.iloc[ind])

$125 | Francisco | train | strawberry
$150 | Erica | football | peach
$175 | Joey | golf ball | raspberry
$200 | Zachary | beach chair | chocolate
$225 | Heidi | sports car | banana


In [11]:
from typing import List, Union
from pydantic import BaseModel, Field

class ConstraintModel(BaseModel):
    constraints_with_reasoning: List[str]


def get_gemini_response(query=None):
    
    if query is None:
        return query
    
    system_prompt = f"""
                        You are an expert AI assistant for analyzing logic grid puzzles. Your task is to systematically translate puzzle clues into logical exptressions.
                        Analyze each clue and categories input to frame a sequence of logical expressions.

                        Remember to carefully comprehend each clue step by step and translate them to logical expressions
                        Retun ONLY a sequence of logical constraints.

                        """


    agent = Agent(system_prompt=system_prompt, agent_name="csp_agent", json_schema=ConstraintModel)
    #user_query= f"""{ques}\n\n{clues}\n\n{categories}"""

    gemini_response = agent.perform_action(query)

    return gemini_response


In [12]:
def get_groq_response(agent_1_response, question, feedback=None):

    query = """"""

    system_prompt = """You are an expert at solving logical grid puzzles. For each puzzle:
    1. Identify all categories and items
    2. Apply each constraint methodically
    3. Present your solution in a clear table format
    4. List only the key deductive steps with minimal explanation
    Keep your reasoning crisp and focused on critical logical inferences.
                    """
    
    ques, categories, clues = process_question(question)

    cat_str = """"""
    for cat in categories:

        cat_str += f"""{cat}: {", ".join(categories[cat])}\n"""


    system_prompt

    query +=  agent_1_response + "\n" 
    query += "Categories: \n" + cat_str + "\n"


    if feedback is not None:
        if isinstance(feedback, dict):
            for index, key in enumerate(feedback):
                if index == 0:
                    query += f"Feedback: \n {key}: \n" 
                    query += "\n".join(feedback[key]) + "\n"

                else:
                    query += f"{key}: {feedback[key]}\n"

    return_seq = question.split("Final Answer:")[1]

    query += return_seq + "\n"

    completion = client.chat.completions.create(
        model="deepseek-r1-distill-llama-70b",
        messages=[{
                "role": "user",
                "content": query

            },
            {"role": "system", "content": system_prompt}],
        temperature=0,
        max_completion_tokens=5000,
        top_p=1,
        stream=True,
        stop=None,
    )

    groq_deepseek_response = ""
    for chunk in completion:
        text = chunk.choices[0].delta.content or ""
        groq_deepseek_response += text

    return groq_deepseek_response

In [13]:
from typing import List, Union
from pydantic import BaseModel, Field

class RewardModel(BaseModel):
    feedback: List[str]
    reward: int


def validate_solution_agent_response(gemini_response, deepseek_response, question):

    query = """"""
    ques, categories, clues = process_question(question)

    system_prompt = """You are the tutor. Look at the student's solution path, check how well it actually solves the problem and follows all constraints, then:
    Give feedbacks and short hints to make the student progress toward the solution. Quantify the accuracy of solving with a score between -100 (completely off-track) and 100 (perfect).
    Speak directly to the student and use this format:
    [Feedback/Summary/Analysis]
    Score: [-100 to 100]"""

    # query += "Problem: \n" + ques + "\n"
    # query += "Clues: \n" + "\n".join(clues) + "\n"
    query += "Categories: \n"

    
    for category in categories:
        item_ = ",".join(categories[category])
        query += f"{category}: {item_}"

    
    query += "Logical Constraints Generated: \n" + gemini_response + "\n"
    query += "Solution from the solution agent: \n" + deepseek_response + '\n'


    reward_agent = Agent(system_prompt=system_prompt, agent_name="reward_agent", json_schema=RewardModel)

    reward_agent_response = reward_agent.perform_action(user_query=query)

    return reward_agent_response

In [13]:
# from typing import List, Union
# import instructor
# from pydantic import BaseModel, Field

# class RewardModel(BaseModel):
#     step_by_step_reasoning: List[str]
#     reward: int


# def validate_constraint_agent_response(gemini_response, deepseek_response, question):

#     query = """"""
#     ques, categories, clues = process_question(question)

#     system_prompt = """Assess constraint translation quality on a scale from -100 to 100 using precise evaluation criteria.
#     Your score should measure how faithfully each original clue is represented in its corresponding logical constraint.
#     Award higher points when constraints fully capture the clue's intent and penalize when significant aspects are overlooked or mistranslated.
#     Provide feedback in the following format:
#     [Step-by-step reasoning for the reward score]
#     Score: [Strictly provide an integer reward score between -100 and 100]"""

#     query += "Problem: \n" + ques + "\n"
#     query += "Clues: \n" + "\n".join(clues) + "\n"
#     query += "Categories: \n"

    
#     for category in categories:
#         item_ = ",".join(categories[category])
#         query += f"{category}: {item_}"

    
#     query += "Logical Constraints Generated: \n" + gemini_response + "\n"

#     client = instructor.from_groq(Groq(), mode=instructor.Mode.JSON)

#     completion = client.chat.completions.create(
#             model="deepseek-r1-distill-llama-70b",
#             response_model=RewardModel,
#             messages=[
#                         {"role": "system", "content": system_prompt},
#                         {"role": "user", "content": query}
#     ],
#             temperature=0,
#             max_completion_tokens=30000,
#             top_p=1,
#             stream=True,
#             stop=None,
#         )

#     groq_deepseek_response = ""
#     for chunk in completion:
#         text = chunk.choices[0].delta.content or ""
#         groq_deepseek_response += text

#     return groq_deepseek_response



In [14]:
import json
from tqdm import tqdm
import traceback
def invoke_agentic_flow(df_question):

    max_retries = 5

    groq_response = None
    final_response = None
    feedback = None
    
    ques, categories, clues = process_question(df_question)
    user_query= f"""{ques}\n\n{clues}\n\n{categories}"""
    
    agent_1_response = get_gemini_response(query=user_query)

    try:
        for try_ in tqdm(range(max_retries)):

            if try_ == 0:
            
                groq_response = get_groq_response(agent_1_response=agent_1_response, question=df_question, feedback=feedback)
                reward = validate_solution_agent_response(gemini_response=agent_1_response, deepseek_response=groq_response, question=df_question)
            
            else:
                
                groq_response = get_groq_response(agent_1_response=agent_1_response, question=df_question, feedback=feedback)
                reward = validate_solution_agent_response(gemini_response=agent_1_response, deepseek_response=groq_response, question=df_question)
                print(reward)
            
            dict_ = json.loads(reward)
            if int(dict_["reward"])  >= 90:
                final_response = groq_response
                break
            else:
                feedback = dict_
                continue
            
    except Exception as e:
        print("Exception:", traceback.format_exc(e))
        return final_response

    final_response = groq_response
    final_reward = int(dict_["reward"])
    return final_reward, final_response
    

In [15]:
import pandas as pd
df = pd.read_csv(r"./data/GridPuzzle_processed.csv")
# df = df[df["id"].isin([13, 821, 687, 718, 350, 1084, 1677, 1437, 1597, 1935, 1020, 1495, 2893, 2208, 2700, 2788, 3720, 3936, 3642, 3659, 4465, 4658, 4087, 4861, 4140, 4464, 4848, 3954, 5740, 5675, 5212, 4976, 5889, 5659, 5325, 5585, 6843, 6292, 6045, 6432, 6823, 5923, 6778, 6997, 7685, 7865, 7183, 7389, 7866, 7188, 7837, 8712, 7959, 8356, 8142, 7993, 8519, 9106, 9719, 9139, 9487, 8853, 8988, 9043, 9497, 9596, 9965, 10037, 10380, 10100, 10450, 10440, 9976, 10251, 11592, 11097, 11022, 11603, 11546, 11748, 11263, 11367, 11407, 11304, 11321, 11923, 12703, 12356, 12535, 12172, 12133, 12395, 12818, 13003, 12900, 13407, 13198, 12782, 13051, 12980])]
# df.head(1)

In [16]:
path = "./agentic_v2_responses"

if not os.path.exists(path):
    os.mkdir(path)

In [ ]:
from tqdm import tqdm
import time
for i,row in tqdm(df.iterrows()):

    if i >= 9 and i<= 100:
    
        id_ = row["id"]
        question = row["question"]

        reward_, final_response  = invoke_agentic_flow(question)
        # print()
        # print(final_response)
        tag = 'table_miss'

        if reward_ >= 75:
            tag = "success"

        with open(os.path.join(path, f"{tag}_{id_}_response.txt"), "w") as f:
            f.write(final_response)
        

0it [00:00, ?it/s]

{
  "feedback": [
    "You're getting bogged down in assigning numbers too early. Focus on deducing relationships between players and positions first. You've correctly identified that Edward Ellis and the placekicker both played Kurt Koch + 2 games, leading to a conflict if they are different people. Explore the implications of them being the same person. Also, consider clue 5, which you haven't fully utilized yet. It gives a direct link between the running back and either Gordon Gregg or the person who played 4 games. Try to integrate this information into your deductions.",
    "Hint: If Edward Ellis is the placekicker, what does that tell you about the other positions and the number of games they played?"
  ],
  "reward": -10
}


 40%|████      | 2/5 [00:33<00:50, 16.68s/it]
10it [00:35,  3.52s/it]

{
  "feedback": [
    "Your reasoning is excellent and you've successfully solved the puzzle! You meticulously went through each clue and constraint, deducing the correct assignments. The final solution is accurate and well-explained.",
    "Score: [100]"
  ],
  "reward": 100
}


{
  "feedback": [
    "Your reasoning is very thorough and you've correctly identified the relationships between the owners and the months. You've also correctly assigned the dog breeds to the owners. However, you seem to be getting lost in the details and re-evaluating the same scenarios multiple times. Focus on the core constraints and try to build the solution step by step. You are on the right track, but try to be more concise in your reasoning.",
    "The final mapping is correct, but the reasoning could be more precise. Try to avoid redundant checks and focus on building the solution systematically."
  ],
  "reward": 80
}


{
  "feedback": [
    "You're making good progress in laying out the information and trying different combinations. However, you seem to be getting stuck in loops. A key observation is that the \"+2 months\" and \"-1 month\" constraints are relative to the *month* the owner is assigned to, not necessarily a fixed starting point. Also, remember that February, March, and April each own a puppy, while January owns an adult dog. This should help narrow down the possibilities for dog breeds associated with each month. Keep trying to link the owners, months, and dog breeds together based on these constraints.",
    "Try focusing on the implications of Rosie owning a Black Lab and the timing constraint associated with it. Where could Rosie be placed, and what does that imply for the Black Lab's month? Also, consider the implications of Violet owning a Boxer. Does this give you any direct links to months or puppy/adult status?"
  ],
  "reward": 30
}


In [58]:
print(df["answer"].iloc[256])

first | vanity | Hannah | Yorba Linda
second | enchantress | Ollie | Miranda
third | red cascade | Casey | Plainfield
fourth | lafter | Francine | Athens
fifth | perle d'or | Lydia | Vail
sixth | quietness | Kristin | Lee Vining


In [9]:
import re
from pprint import pprint
incorrect = []
for file in os.listdir(path):
    flag = -1
    id_ = file.split("_")[1]
    with open(os.path.join(path, file), "r") as f:
        lines = f.readlines()
    
    lines_rev = lines[::-1]

    breaker = len(lines_rev) // 2
    for line_count, line in enumerate(lines_rev):

        if line_count > breaker and flag == -1:
            break
        
        if re.search(r"\|", line) is not None:
            flag = 1
            break
        else:
            continue    
        
    if flag == -1:
        incorrect.append(file)


print("Number of incorrect files:", len(incorrect))
print()
print()
pprint(incorrect)

Number of incorrect files: 8


['df_1345_response.txt',
 'df_2300_response.txt',
 'df_244_response.txt',
 'df_3458_response.txt',
 'df_3642_response.txt',
 'df_3957_response.txt',
 'df_4624_response.txt',
 'df_688_response.txt']


In [54]:
output = {}
for file in os.listdir(path):
    flag = -1
    id_ = file.split("_")[1]
    
    with open(os.path.join(path, file), "r") as f:
        lines = f.readlines()
    
    lines_rev = lines[::-1]

    flag = -1
    ckpt = -1
    reqd_lines = []
    for line_count, line in enumerate(lines_rev):

        if flag == 1 and ckpt == 1:
            break

        
        if re.search(r"\|", line) is not None:
            if ckpt == -1:
                ckpt = 1
            reqd_lines.append(line)
            
        

        
        elif ckpt == 1 and re.search(r"\|", line) is None:
            ckpt =-1
            flag = 1
    
    output[id_] = reqd_lines[::-1]



        
print(len(output))

100


In [20]:
for key in output:
    if len(output[key]) >= 6:
        pprint(output[key])

['| Year  | Island   | Culture    |\n',
 '|-------|----------|------------|\n',
 "| 1754  | Zafet    | Manika'i   |\n",
 '| 1761  | Fushil   | Hakili     |\n',
 '| 1768  | Jujihm   | Kukani     |\n',
 "| 1775  | Teuz     | Holo'oka   |"]
['The Senator | ? | 85\n',
 '| Height (ft) | Tree           | Age (years) |\n',
 '|---------------|----------------|-------------|\n',
 "| 144           | Evan's Fir     | 94          |\n",
 "| 147           | Zeke's Spruce  | 80          |\n",
 '| 150           | The Senator    | 85          |\n',
 '| 153           | Marsh Tree     | 96          |']
['| Year  | Windmill      | Family      |\n',
 '|-------|--------------|-------------|\n',
 '| 1683  | Doesmolen    | Meijer      |\n',
 '| 1706  | Vlietmolen   | Bakker      |\n',
 '| 1729  | Grosmolen    | De Vries    |\n',
 '| 1752  | Zemelmolen   | Van Dijk    |\n']
['June | Hannah | Juniper Springs  \n',
 '| Month   | Witness   | Location          |\n',
 '|----------|-----------|--------------------|\

In [27]:
output

{'1020': ['$550 | Honwa | 290  \n',
  '$575 | Cober | 5000  \n',
  '$600 | Banion | 5  \n',
  '$625 | Dayero | 520'],
 '1042': ['2009 | Barbara | Pekingese\n',
  '2006 | Fernando | Great Dane  \n',
  '2007 | Chester | Beagle  \n',
  '2008 | Elsie | Maltese  \n',
  '2009 | Barbara | Pekingese'],
 '1084': ['2007 | Max Fusion | Lyle Lucas  \n',
  '2008 | Green Avenger | Hal Houston  \n',
  '2009 | Criminal Bane | Peter Powers  \n',
  '2010 | Ultra Hex | Tim Trevino'],
 '1234': ['5 | Iggy | 115  \n',
  '10 | Sparky | 120  \n',
  '15 | Arthur | 136  \n',
  '20 | Carlos | 140  \n'],
 '1250': ['850 AD | Budel | 9500  \n',
  '100 AD | Hatgare | 5000  \n',
  '350 AD | Sameos | 7500  \n',
  '600 AD | Oiracha | 4500  \n',
  '850 AD | Budel | 9500'],
 '1309': ['| Year  | Island   | Culture    |\n',
  '|-------|----------|------------|\n',
  "| 1754  | Zafet    | Manika'i   |\n",
  '| 1761  | Fushil   | Hakili     |\n',
  '| 1768  | Jujihm   | Kukani     |\n',
  "| 1775  | Teuz     | Holo'oka   |"]

In [32]:
gt = {}

count = 0 
for i, row in df.iterrows():

    if count >= 100:
        break

    if output.get(str(row["id"])) is not None:

        gt[str(row["id"])] = row["answer"]
        count += 1


print(len(gt))


    

100


In [34]:
count = 0
for key in gt:
    if output.get(key) is not None:
        count += 1

print(count)

100


In [38]:
for key in gt:
    print(gt[key])
    print()
    print("".join(output[key]))
    break

1984 | Annata Branco | gewurztraminer
1988 | Ece Suss | pinot noir
1992 | Bianca Flaux | merlot
1996 | Vendemmia | riesling

1984 | Annata Branco | Gewurztraminer  
1988 | Ece Suss | Pinot Noir  
1992 | Bianca Flaux | Merlot  
1996 | Vendemmia | Riesling


In [44]:
def process_pred(pred_string):
    gt_dict = {}
    lines = pred_string.split("\n")
    for line in lines:

        if len(line.split("|")) >= 3:
        
            entities = line.split("|")
            cleaned_entities = []
            for entity in entities:
                cleaned_entity = entity.strip().strip("\n")
                cleaned_entity = entity.lower()
                cleaned_entities.append(cleaned_entity)
        
            for ent in cleaned_entities[1:]:
                gt_dict[ent] = cleaned_entities[0]
    
    return gt_dict

In [43]:
def process_gt(gt_string):
    gt_dict = {}
    lines = gt_string.split("\n")
    for line in lines:
        
        entities = line.split("|")
        cleaned_entities = []
        for entity in entities:
            cleaned_entity = entity.strip().strip("\n")
            cleaned_entity = entity.lower()
            cleaned_entities.append(cleaned_entity)
        
        for ent in cleaned_entities[1:]:
            gt_dict[ent] = cleaned_entities[0]
    
    return gt_dict

In [50]:
def get_hard_predictions(gt, pred):

    gt_dict = process_gt(gt)
    pred_dict = process_pred(pred)
    flag = True
    
    print("pred_dict", pred_dict)
    print()
    print("gt_dict", gt_dict)



    for key in gt_dict:

        if gt_dict[key] != pred_dict.get(key):
            flag=False
            break
    
    if flag:
        return 1

    else:
        return 0

In [42]:
incorrect_ids = [val.split("_")[1] for val in incorrect]

In [51]:
hard_predictions = {}
for key in gt:
    
    if key not in incorrect_ids:
        gt_val = gt.get(key)
        pred_val = "\n".join(output.get(key))
        
        if gt_val is not None and pred_val is not None:
            predicted_metric = get_hard_predictions(gt_val, pred_val)
            hard_predictions[key] = predicted_metric
    

values_sum = sum(list(hard_predictions.values()))


metric = values_sum / len(hard_predictions)

print(values_sum, len(hard_predictions), metric)







pred_dict {' annata branco ': '1984 ', ' gewurztraminer  ': '1984 ', ' ece suss ': '1988 ', ' pinot noir  ': '1988 ', ' bianca flaux ': '1992 ', ' merlot  ': '1992 ', ' vendemmia ': '1996 ', ' riesling': '1996 '}

gt_dict {' annata branco ': '1984 ', ' gewurztraminer': '1984 ', ' ece suss ': '1988 ', ' pinot noir': '1988 ', ' bianca flaux ': '1992 ', ' merlot': '1992 ', ' vendemmia ': '1996 ', ' riesling': '1996 '}
pred_dict {' altissimo ': 'fourth ', ' hannah': 'fourth ', ' calocarpa ': 'first ', ' nellie  ': 'first ', " perle d'or ": 'second ', ' tricia  ': 'second ', ' bon silene ': 'third ', ' renee  ': 'third '}

gt_dict {' bon silene ': 'first ', ' renee': 'first ', " perle d'or ": 'second ', ' tricia': 'second ', ' calocarpa ': 'third ', ' nellie': 'third ', ' altissimo ': 'fourth ', ' hannah': 'fourth '}
pred_dict {' score (%) ': '', ' a.i.      ': '', ' institution ': '', '': '', '-----------': '', '-------------': '', ' 55        ': '', ' archimedes': '', ' cit         ': '',

In [ ]:
manual_validated = []
for key in gt:
    if key not in incorrect_ids:

        gt_ = gt[key]
        out_ = "".join(output[key])

        print("predicted", out_)
        print()
        print("gt", gt_)

        if input() == "1":
            manual_validated.append(1)

     

predicted 1984 | Annata Branco | Gewurztraminer  
1988 | Ece Suss | Pinot Noir  
1992 | Bianca Flaux | Merlot  
1996 | Vendemmia | Riesling

gt 1984 | Annata Branco | gewurztraminer
1988 | Ece Suss | pinot noir
1992 | Bianca Flaux | merlot
1996 | Vendemmia | riesling
predicted fourth | altissimo | Hannah
first | calocarpa | Nellie  
second | perle d'or | Tricia  
third | bon silene | Renee  
fourth | altissimo | Hannah

gt first | bon silene | Renee
second | perle d'or | Tricia
third | calocarpa | Nellie
fourth | altissimo | Hannah


In [ ]:
import torch

In [ ]:
checkpoint = torch.load(os.path.join(path, f'style_gan_checkpoint_res{4*2**step}.pth'))

alpha = checkpoint['alpha']


# Method 1: Save alpha using torch.save()
torch.save(alpha, 'alpha.pth')

# Method 2: Save as a dictionary with torch.save()
torch.save({'alpha': alpha}, 'alpha_dict.pth')

# Method 3: If alpha is a simple scalar value, you can use standard Python
import json
with open('alpha.json', 'w') as f:
    json.dump({'alpha': float(alpha)}, f)  # Convert to float in case it's a tensor

# Method 4: Simple text file (if alpha is a scalar)
with open('alpha.txt', 'w') as f:
    f.write(str(float(alpha)))